# FRETraj Tutorial - Modeling with FRET restraints

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import mdtraj as md
import fretraj as ft

In [2]:
import importlib

In [3]:
#importlib.reload(ft.restraints)

### Accessible contact volume

Calculate ACVs for donor and acceptor dye on an **energy minimized and fully solvated** biomolecule.
> Note: the generated .gro file must first be converted to a .pdb file using `gmx trjconv`

In [4]:
labels = ft.cloud.labeling_params('DNA_parameters.json')

In [5]:
struct = md.load_pdb('DNA.pdb')
struct_nosolv = struct.remove_solvent()

In [6]:
don = ft.cloud.Volume(struct_nosolv, '1-DT20-P', labels)
acc = ft.cloud.Volume(struct_nosolv, '1-DC44-O3\'', labels)

Compute a FRET trajectory from the ACVs

In [7]:
fret_traj = ft.cloud.FRET_Trajectory(don, acc, 'default param.', labels)

Create a Plumed object from the loaded structure and the ACVs. Look for neighboring phosphorus atoms within a range of 1.5 nm. 

In [22]:
plumed = ft.restraints.Plumed(struct, [don, acc], selection='name P', cutoff=1.5)

Set your desired FRET value that you want the simulation converge to. The corresponding $\langle R_{DA,E}\rangle$ will be computed from the $\langle E\rangle$. The $\langle R_{mp}\rangle$ will be calculated from $\langle E\rangle$ using a third order polynomial (Kalinin et al., *Nat. Methods*, 2012)

In [23]:
targetFRET = 0.5
mean_R_DA_E = ft.fret.mean_dist_DA_fromFRET(None,None,targetFRET,54)
targetRmp = ft.fret.R_DAE_to_Rmp(mean_R_DA_E)

Compile a plumed input file for the MD simulation

In [19]:
plumed.write_plumed('out/plumed_DT20-DC44.dat', targetRmp, 100, 100)

To illustrate the restaints save a visualization skript for VMD or PyMOL which can be run with either `vmd_vis -c filename.gro -x trajectory.xtc -v vis.vmd` or `pymol_vis -c filename.gro -x trajectory.xtc -v vis.py`

In [107]:
plumed.write_vmd('out/vis.vmd')

In [77]:
plumed.write_pymol('out/vis.py')

Save force field and topology files files of the mean position pseudoatoms 

In [110]:
plumed.write_pseudo('out/MP.pdb', 'out/MP.itp')

(1) Copy the "MP.itp" file into your force field directory

(2) Add the following lines to your "topology.top" file:
; Include topology for dye mean position
#include "<path/to/forcefield>/MP.itp"

#ifdef POSRES
[ position_restraints ]
1     1  1000  1000  1000
#endif
(3) Add the following line to your "atomtypes.atp" file in the force field directory:
MP        0.00000 ;

(4) Add the following lines to your "ffnonbonded.itp" file in the force field directory:
; Dummy mass for dye mean position
MP          35      79.90    0.0000  A   4.64693e-01  2.45414e-01


In [55]:
don.save_mp('out/p19_D.dat', units='nm', format='plain')
acc.save_mp('out/p19_A.dat', units='nm', format='plain')

First, backup the original .gro file. Then, insert the mean position coordinates given in **MP.pdb** at positions specified by **DNA_D.dat** and **DNA_A.dat** 

```
mv DNA.gro DNA_original.gro
gmx insert-molecules -ci MP.pdb -f DNA_original.gro -o DNA.gro -ip out/DNA_D.dat -replace water |& tee tmp_insert.dat
gmx insert-molecules -ci MP.pdb -f DNA.gro -o DNA.gro -ip out/DNA_A.dat -replace water |& tee -a tmp_insert.dat
```

Run the following command will tell you how many solvent molecules have been replaced by the MP pseudoatoms and how to update the topology. 

```
awk '$1 == "Replaced" {sum += $2}; END {print "\nIn your topology.top file under the section [ molecules ] do the following:\n(1) decrease the number of solvent molecules by " sum "\n(2) add the following line:\nMP        2"}' tmp_insert.dat && rm tmp_insert.dat
```